In [1]:
import import_ipynb
from tweet_prepossesssing_and_clustering import is_english,extract_hashtag,remove_stop_words,lemmotize,tokenize,generate_bow,count_dict,tf_idf_vector,one_hot_encoding_vector,cosine_similarity,jaccard_similarity,dice_similarity,k_means_clustering,most_frequent_words,generate_word_cloud,iterative_clustering
from chainning_and_analysis import calculate_silhouette_scores,calculate_cluster_weights,event_clusters_filter,generate_clusters,write_clusters_to_text_and_hashtags,read_clusters_from_file,read_hashtags_from_file,centroid,calculate_similarity,textual_similarity,create_bipartite_graph,apply_hungarian_method,create_cluster_chains,get_folder_names,write_cluster_chains,analyze_sentiment,plot_pie_chart,generate_emoji_image,generate_word_cloud,analyze_and_visualize_clusters

importing Jupyter notebook from tweet_prepossesssing_and_clustering.ipynb
importing Jupyter notebook from chainning_and_analysis.ipynb


In [2]:
from sklearn.metrics import silhouette_score
import numpy as np
import emoji
from math import sqrt
import pandas as pd
import re
import networkx as nx
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import pos_tag
import string
from langdetect import detect
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from collections import Counter
import operator
from textblob import TextBlob
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch
from sklearn.metrics import pairwise_distances
from collections import defaultdict
from scipy.optimize import linear_sum_assignment
import emojis
from PIL import Image, ImageDraw, ImageFont

In [3]:
folder_path = os.getcwd()
folder_path = os.path.join(folder_path,'output_tweets')
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        print(filename)
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, names=['Text'])
        df = df[df['Text'].apply(is_english)]
        extract_hashtag(df)
        tokenize(df)
        df['filtered_tokens'] = df['Tokenized_Tweets'].apply(remove_stop_words)
        df = df[df['filtered_tokens'].apply(lambda x: len(x) > 0)]
        df.reset_index(drop = True,inplace = True)
        bag_of_words = generate_bow(df['filtered_tokens'])
        vector = tf_idf_vector(df['filtered_tokens'].tolist(),bag_of_words.keys())
        vector = np.array(vector)
        cluster,no_of_clusters = iterative_clustering(vector,0.1)
        
        cluster_qualities = calculate_silhouette_scores(np.array(vector),np.array(cluster))
        cluster_weights = calculate_cluster_weights(vector, cluster)
        event_clusters = event_clusters_filter(cluster_qualities,cluster_weights,0.7,6.0)
        event_clusters_tweets,event_clusters_hashtags = generate_clusters(list(df['filtered_tokens']),cluster,event_clusters,list(df['Hashtag']))
        no_of_clusters = len(event_clusters)
        if no_of_clusters < 1:
            continue
        word_sets = most_frequent_words(event_clusters_tweets,no_of_clusters)
        folder_name = filename[:len(filename) - 4]
        folder_name = os.path.join('event_tweets',folder_name)
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        file_path = os.getcwd()
        file_path = os.path.join(file_path,folder_name)
        event_cluster_file = os.path.join(file_path,'event_clusters.txt')
        event_hashtags_file = os.path.join(file_path,'event_hashtags.txt')
        write_clusters_to_text_and_hashtags(event_clusters_tweets,event_clusters_hashtags,event_cluster_file,event_hashtags_file)
        for i in range(no_of_clusters):
            print(sorted(word_sets[i].items(),key = operator.itemgetter(1),reverse = True)[:10])
            generate_word_cloud(word_sets[i], i, os.path.join(file_path,f"img{i}"))

In [4]:
folder_path = os.path.join(os.getcwd(),'event_tweets')
timestamps = get_folder_names(folder_path)

In [5]:
resulting_chains = create_cluster_chains(timestamps,folder_path,0.5)

2022-10-23_08-00_2022-10-23_08-05 and 2022-10-23_08-05_2022-10-23_08-10
2022-10-23_08-05_2022-10-23_08-10 and 2022-10-23_08-10_2022-10-23_08-15
2022-10-23_08-10_2022-10-23_08-15 and 2022-10-23_08-20_2022-10-23_08-25
2022-10-23_08-20_2022-10-23_08-25 and 2022-10-23_08-25_2022-10-23_08-30
2022-10-23_08-25_2022-10-23_08-30 and 2022-10-23_08-30_2022-10-23_08-35
2022-10-23_08-30_2022-10-23_08-35 and 2022-10-23_08-35_2022-10-23_08-40
2022-10-23_08-35_2022-10-23_08-40 and 2022-10-23_08-40_2022-10-23_08-45
2022-10-23_08-40_2022-10-23_08-45 and 2022-10-23_08-45_2022-10-23_08-50
2022-10-23_08-45_2022-10-23_08-50 and 2022-10-23_08-50_2022-10-23_08-55
2022-10-23_08-50_2022-10-23_08-55 and 2022-10-23_08-55_2022-10-23_09-00
2022-10-23_08-55_2022-10-23_09-00 and 2022-10-23_09-00_2022-10-23_09-05
2022-10-23_09-00_2022-10-23_09-05 and 2022-10-23_09-05_2022-10-23_09-10
2022-10-23_09-05_2022-10-23_09-10 and 2022-10-23_09-10_2022-10-23_09-15
2022-10-23_09-10_2022-10-23_09-15 and 2022-10-23_09-15_2022-10-2

In [6]:
def create_clusters_dict(folder_path):
    clusters_dict = {}
    folders = sorted(os.listdir(folder_path))
    for i in range(len(folders)-1):
        current_folder = folders[i]
        next_folder = folders[i+1]

        current_file_path = os.path.join(folder_path, current_folder, 'event_clusters.txt')
        next_file_path = os.path.join(folder_path, next_folder, 'event_clusters.txt')

        current_clusters = read_clusters_from_file(current_file_path)
        next_clusters = read_clusters_from_file(next_file_path)

        clusters_dict[current_folder] = current_clusters
        clusters_dict[next_folder] = next_clusters

    return clusters_dict

In [7]:
clusters_dict = create_clusters_dict(os.path.join(os.getcwd(),'event_tweets'))

In [8]:
write_cluster_chains(resulting_chains,os.getcwd(),clusters_dict)

In [10]:
cluster_chains_folder = os.path.join(os.getcwd(),'cluster_chains')
output_folder = os.path.join(os.getcwd(),'Sentiment_analysis')
analyze_and_visualize_clusters(cluster_chains_folder, output_folder)